<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/TAPT_Amazon_Classification_seq_bn_TAPT_pretrained_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This runs the Amazon Helpfulness Classification on the TAPT helpfulness trained full model.

In [1]:
from time import time
start_time = time()

In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install transformers[torch]
!pip install datasets
!pip install scikit-learn
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.3 MB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")



In [6]:
from sklearn.metrics import accuracy_score,  f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [7]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_helpfulness"
dataset = load_dataset(dataset_name)
dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 115251
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [8]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
#Label Settings
id2label = {0: "unhelpful", 1: "helpful"}
label2id = {"unhelpful": 0, "helpful": 1}

In [10]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'heads.default.3.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
load_name = f"BigTMiami/pretrain_tapt_seq_bn_adpater"

# Load Pretrained adapter without head
loaded_adapter_name = model.load_adapter(load_name, with_head=False)

# Add head for classification modeling
model.add_classification_head(
    loaded_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
model.train_adapter(loaded_adapter_name)

print(f"Classification Adapter Name: {loaded_adapter_name}")

adapter_hub_name = "tapt_seq_bn_amazon_helpfulness_classification_adapter_v2"

(…)b/master/dist/v2/index/roberta-base.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

head_config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

Classification Adapter Name: tapt_seq_bn


In [12]:
summary = model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
tapt_seq_bn              bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="tapt_seq_bn_amazon_helpfulness_classification_model_v2",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.0001, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10, # NEED TO TRAIN FOR REAL TEST
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=1, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    load_best_model_at_end=True, # Set to false - we want the last trained model like the paper
    metric_for_best_model="f1_macro", # Is default
    # torch_compile=True,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
)

In [14]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["test"],
    eval_dataset=dataset["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [15]:
results = trainer.train()
print(results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.338400,0.330842,0.858600,0.673888
2,0.324500,0.325626,0.865200,0.671852
3,0.325800,0.340772,0.867400,0.646354
4,0.330900,0.314975,0.867800,0.652682
5,0.292000,0.322595,0.869200,0.678703
6,0.275600,0.338429,0.868800,0.649793
7,0.258400,0.348877,0.865400,0.694643
8,0.275800,0.353950,0.864000,0.694977
9,0.247600,0.353960,0.866800,0.668790
10,0.230300,0.368583,0.866200,0.654184


TrainOutput(global_step=15630, training_loss=0.28822878766197163, metrics={'train_runtime': 4401.9398, 'train_samples_per_second': 56.793, 'train_steps_per_second': 3.551, 'total_flos': 6.29913826630344e+16, 'train_loss': 0.28822878766197163, 'epoch': 10.0})


In [16]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.3539504408836365, 'eval_accuracy': 0.864, 'eval_f1_macro': 0.6949774265353131, 'eval_runtime': 37.8501, 'eval_samples_per_second': 132.1, 'eval_steps_per_second': 8.269, 'epoch': 10.0}


In [17]:
model.push_adapter_to_hub(
    adapter_hub_name,
    loaded_adapter_name,
    datasets_tag=dataset_name
)
print(f"Pushed {adapter_hub_name}")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Pushed tapt_seq_bn_amazon_helpfulness_classification_adapter_v2


In [ ]:
trainer.push_to_hub()

events.out.tfevents.1713312292.486e13d64ef4.279.1:   0%|          | 0.00/463 [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

events.out.tfevents.1713307851.486e13d64ef4.279.0:   0%|          | 0.00/33.8k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/tapt_seq_bn_amazon_helpfulness_classification_model_v2/commit/adc81b70c1fcd44056946a344ec94a39280f72f9', commit_message='End of training', commit_description='', oid='adc81b70c1fcd44056946a344ec94a39280f72f9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
end_time = time()
total_time = end_time - start_time
print(f"Total Script Runtime: {total_time / 60:.0f}  minutes OR {total_time:.0f}  seconds")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()
